In [105]:
import pandas as pd
counter_away = 0
counter_home = 0

In [124]:
import os

path = "/Users/Deen/Desktop/Neural_Nets/Final_Project/Monthly_Stats_CSV/"

files = os.listdir(path)

large_df = []

# Go through every file in the Monthly_Stats_CSV folder
for file in files:

    # Break the file down into its components
    file_list = file.split("-")
    month = int(file_list[1].split("_")[1].split(".")[0])

    # Here, year is defined by the earlier year in the season. Example: 2009-10 is "09"
    year = file_list[0][2:]
    year_str = "20" + year + "-" + str(int(year)+1)

    # Using the information from the file name in Monthly_Stats_CSV, read in the file from Game_Results_CSV
    game_results = pd.read_csv("Game_Results_CSV/" + year_str + "_" + str(month+1) + ".csv")

    # Keep only certain columns and rename those that we do keep
    game_results = game_results[["Visitor/Neutral", "PTS", "Home/Neutral", "PTS.1", "Attend."]]
    game_results.rename(columns = {"Visitor/Neutral" : "TEAM", "PTS" : "PTS_AWAY", "PTS.1" : "PTS_HOME", "Attend." : "Attendance"}, inplace = True)

    # Los Angeles Clippers have weird naming, so we fix that:
    game_results["TEAM"] = game_results["TEAM"].str.replace("LA Clippers", "Los Angeles Clippers")
    game_results["Home/Neutral"] = game_results["Home/Neutral"].str.replace("LA Clippers", "Los Angeles Clippers")

    # Read in the Monthly_Stats file
    monthly_stats = pd.read_csv(path+file)
    monthly_stats["TEAM"] = monthly_stats["TEAM"].str.replace("LA Clippers", "Los Angeles Clippers")

    # Drop the unnecessary columns
    cols = [0, 1, 3, 4, 5, 7]
    cols_2 = list(range(29,55))
    for column in cols_2:
        cols.append(column)
    monthly_stats.drop(monthly_stats.columns[cols], axis=1, inplace=True)

    # Join the stats for the away team
    combined = game_results.merge(monthly_stats, how="outer", on=["TEAM"])

    # Only need to define the dictionary for renaming the columns once, doing it multiples times would waste time
    if counter_away == 0:
        away_names = {}

        for index in range(combined.shape[1] - 5):
            column = combined.iloc[:, index + 5]
            away_names[column.name] = column.name + "_away"
        counter_home += 1

    # Rename the columns to reflect that they belong to the away team
    combined.rename(columns = away_names, inplace = True)

    # Join the stats for the home team
    combined.rename(columns = {"TEAM" : "AWAY"}, inplace = True)
    combined.rename(columns = {"Home/Neutral" : "TEAM"}, inplace = True)
    final_df = combined.merge(monthly_stats, how="outer", on=["TEAM"])

    # Only need to define the dictionary for renaming the columns once, doing it multiples times would waste time
    if counter_home == 0:
        home_names = {}

        for index in range(final_df.shape[1] - 27):
            column = final_df.iloc[:, index + 27]
            home_names[column.name] = column.name + "_home"
        counter_home += 1

    # Rename the columns to reflect that they belong to the home team
    final_df.rename(columns = home_names, inplace = True)

    # Create a column that indicates whether the home team won or lost
    final_df["Home_Win"] = final_df["PTS_HOME"] > final_df["PTS_AWAY"]

    large_df.append(final_df)



In [129]:
game_stats = pd.concat(large_df)

#game_stats.drop(game_stats.columns[[0]], axis = 1, inplace = True)

In [118]:
game_stats

,AWAY,PTS_AWAY,TEAM,PTS_HOME,Attendance,WIN%_away,PTS_away,FGM_away,FGA_away,FG%_away,...,REB_home,AST_home,TOV_home,STL_home,BLK_home,BLKA_home,PF_home,PFD_home,+/-_home,Home_Win
0,Memphis Grizzlies,109.0,Atlanta Hawks,112.0,11513.0,0.400,100.1,39.2,83.5,47.0,...,39.9,24.3,13.0,5.7,4.7,3.7,19.1,18.9,2.1,True
1,Orlando Magic,81.0,Atlanta Hawks,91.0,16275.0,0.800,99.6,36.0,76.1,47.3,...,39.9,24.3,13.0,5.7,4.7,3.7,19.1,18.9,2.1,True
2,Charlotte Bobcats,85.0,Atlanta Hawks,90.0,15006.0,0.429,97.7,36.8,76.7,48.0,...,39.9,24.3,13.0,5.7,4.7,3.7,19.1,18.9,2.1,True
3,Indiana Pacers,83.0,Atlanta Hawks,97.0,14131.0,0.538,99.5,36.9,83.3,44.3,...,39.9,24.3,13.0,5.7,4.7,3.7,19.1,18.9,2.1,True
4,New Jersey Nets,101.0,Atlanta Hawks,116.0,14273.0,0.267,92.9,34.6,77.5,44.6,...,39.9,24.3,13.0,5.7,4.7,3.7,19.1,18.9,2.1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,Oklahoma City Thunder,104.0,Los Angeles Lakers,103.0,18997.0,0.333,91.1,34.5,81.6,42.2,...,42.8,20.2,12.2,7.5,4.5,4.1,21.9,22.7,-6.4,False
229,New Orleans Pelicans,104.0,Los Angeles Lakers,87.0,18997.0,0.429,100.4,38.3,84.6,45.3,...,42.8,20.2,12.2,7.5,4.5,4.1,21.9,22.7,-6.4,False
230,Phoenix Suns,116.0,Los Angeles Lakers,107.0,18997.0,0.500,104.3,38.6,85.7,45.0,...,42.8,20.2,12.2,7.5,4.5,4.1,21.9,22.7,-6.4,False
231,Golden State Warriors,105.0,Los Angeles Lakers,115.0,18997.0,0.867,107.9,40.5,82.7,49.0,...,42.8,20.2,12.2,7.5,4.5,4.1,21.9,22.7,-6.4,True


In [130]:
game_stats.to_csv("game_stats.csv")

print(game_stats.nunique())

AWAY            33
PTS_AWAY        97
TEAM            33
PTS_HOME        90
Attendance    4501
WIN%_away      100
PTS_away       306
FGM_away       134
FGA_away       187
FG%_away       120
3PM_away       130
3PA_away       294
3P%_away       169
FTM_away       134
FTA_away       163
FT%_away       197
OREB_away       90
DREB_away      124
REB_away       149
AST_away       135
TOV_away        88
STL_away        71
BLK_away        64
BLKA_away       58
PF_away         97
PFD_away       103
+/-_away       271
WIN%_home      100
PTS_home       306
FGM_home       134
FGA_home       187
FG%_home       120
3PM_home       130
3PA_home       294
3P%_home       169
FTM_home       134
FTA_home       163
FT%_home       197
OREB_home       90
DREB_home      124
REB_home       149
AST_home       135
TOV_home        88
STL_home        71
BLK_home        64
BLKA_home       58
PF_home         97
PFD_home       103
+/-_home       271
Home_Win         2
dtype: int64
